In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time 
import requests
from bs4 import BeautifulSoup
import urllib.request


driver = webdriver.Edge()

def scrape_style(url, style_name):
    big_iter = 780
    base = 'http://invarquit.cultura.gencat.cat/Cerca/'  
    driver.get(url)
    first_building = driver.find_elements(By.CLASS_NAME, "enlacedestacado")[0] #Do this only once. From here on uses other button
    first_building.click()
    download_images(driver.current_url, style=style_name, big_it=big_iter) #To click trough all images and download each for first 
    next_buttons = driver.find_elements(By.TAG_NAME, 'img') 
    for button in next_buttons:
        if 'endavant' in button.get_attribute('src'):
            if "public" in button.get_attribute('title'):
                next_building = button
    next_building.click() #Go to next building
    contin=True
    while contin:
        big_iter += 1
        next_buttons = driver.find_elements(By.TAG_NAME, 'img')
        for button in next_buttons:
            if 'endavant' in button.get_attribute('src'):
                if "public" in button.get_attribute('title') and not "Off" in button.get_attribute('title'):
                    next_building = button #There is a next building 
                elif "public" in button.get_attribute('title') and "Off" in button.get_attribute('title'): #Last building of style
                    download_images(driver.current_url, style=style_name, big_it=big_iter) #To click trough all images and download each 
                    contin = False   
        if contin:
            next_building.click()
            download_images(driver.current_url, style=style_name, big_it=big_iter) #To click trough all images and download each 


def download_images(url, style, big_it):
    it = 0
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        candid_dupl = soup.find_all(class_ = "taulesDeDins") 
        if len(candid_dupl[0]) == 5:                    #To check if only a single style label is assigned to the building
            img_source = driver.find_element(By.ID, "imatgeACanviar").get_attribute("src")
            urllib.request.urlretrieve(img_source, '{}/{}-{}-{}.jpg'.format(style,style, big_it, it)) #Naming convention
            next_buttons = driver.find_elements(By.TAG_NAME, 'img')
            for button in next_buttons:
                if 'endavant' in button.get_attribute('src'):
                    if not "public" in button.get_attribute('title'):
                        next_img = button
            next_img.click()
            new_img_source = driver.find_element(By.ID, "imatgeACanviar").get_attribute("src")
            while new_img_source != img_source and it < 7: #Download at most 7 images
                it += 1
                urllib.request.urlretrieve(new_img_source, '{}/{}-{}-{}.jpg'.format(style,style, big_it, it))
                next_buttons = driver.find_elements(By.TAG_NAME, 'img')
                for button in next_buttons:
                    if 'endavant' in button.get_attribute('src'):
                        if not "public" in button.get_attribute('title'):
                            next_img = button
                next_img.click()
                img_source = new_img_source
                new_img_source = driver.find_element(By.ID, "imatgeACanviar").get_attribute("src")   
    except:
        pass

style_list = ['baroque', 'contemporary', 'gothic', 'modernism', 'neoclassicism', 'noucentisme', 'renaissance', 'romanesque']
corr_url = ['http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K0JhcnJvYyU%3D', 'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K0RhcnJlcmVzIHRlbmTDqG5jaWVzJQ%3D%3D',
'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K0fDsnRpYyU%3D', 'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K01vZGVybmlzbWUl', 'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K05lb2NsYXNzaWNpc21lJQ%3D%3D',
'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K05vdWNlbnRpc21lJQ%3D%3D', 'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K1JlbmFpeGVtZW50JQ%3D%3D', 
'http://invarquit.cultura.gencat.cat/Cerca/Llista?Consulta=MCU4K1JvbcOgbmljJQ%3D%3D'] #Loop of all links necessary. Can easily be extended.

for idx, style in enumerate(style_list): #Actual command to run and scrape all. Note that everything will take a long time. 
    URL = corr_url[idx]
    scrape_style(URL, style)